In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
from pyspark.sql import functions as F
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 
from pyspark.sql.functions import collect_list, concat_ws

runcycleid = etl_helpers.start_run_cycle("factRequestDocumentsDetails")

os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    today = str(datetime.date.today())

    # last successful run time
    df_existing = spark.sql("SELECT max(TIMESTAMP(runcycleendat)) as runcycleendat from dimruncycle where packagename = 'factRequestDocumentsDetails' and success = 't'")
    df_existing.show()
    maxcreatedate = df_existing.first().runcycleendat
    print(maxcreatedate)
    maxcreatedate_str = maxcreatedate.strftime("%Y-%m-%d %H:%M:%S")

    # temp table/view for new/update requestids
    df_existing = spark.sql(f"""
        SELECT DISTINCT
            foiministryrequestid,
            foirequest_id
        FROM
            foi_mod.foiministryrequests
        WHERE
            created_at > '{maxcreatedate_str}' OR DATE(updated_at) > '{maxcreatedate_str}'
    """)
    df_existing.createOrReplaceTempView("temp_requests")
    df_existing.show()

    if df_existing.count() == 0:
        raise Exception("no changes for today")

    query = f"""MERGE INTO default.factRequestDocumentsDetails dd
        USING (
            SELECT foirequestid
            FROM temp_requests
        ) AS temp
        ON temp.foirequestid = dd.foirequestid
        WHEN MATCHED and dd.sourceoftruth = 'FOIMOD' THEN

        UPDATE 
        SET dd.activeflag = 'N'""";

        
    comma_list_df = df_existing.select(concat_ws(",", collect_list("foiministryrequestid")).alias("comma_list"))
    comma_list = comma_list_df.collect()[0]["comma_list"]

    query = f"""
        select sq1.foiministryrequestid, count, 
        case when requeststatuslabel = 'closed' then count else 0 end as pagesreleased,
        sq2.created_at, sq2.updated_at, sq2.foirequest_id,

        estimatedelectronicpages, estimatedhardcopypages,
        pagecount, dedupepagecount
        from 
        ((select foiministryrequestid, count(*) as count
        from (
        select 
        EXPLODE(
            FROM_JSON(
                pageflag,
                'ARRAY<STRUCT<flagid: INT, page: INT, programareaid: ARRAY<INT>, other: ARRAY<STRING>>>'
            )
            ) as pageflags,
        -- explode(split(pageflag, ',')) as pageflags,
        -- pageflag, 
        dpf.foiministryrequestid from docreviewer.documentpageflags dpf
        join docreviewer.documents d on dpf.documentid = d.documentid
        join docreviewer.documentmaster dm on dm.documentmasterid = d.documentmasterid
        left join docreviewer.documentdeleted dd on dd.filepath || '%' ilike dm.filepath
        where dd.deleted is null or dd.deleted is false
        )
        group by foiministryrequestid ) sq1

        join (WITH ranked AS (
        SELECT *,
                ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version DESC) AS rn
        FROM foi_mod.FOIMinistryRequests fmr
        )
        SELECT ranked.foiministryrequestid, ranked.requeststatuslabel, ranked.created_at, ranked.updated_at, ranked.foirequest_id
        FROM ranked
        WHERE rn = 1 and (foiministryrequestid in ({comma_list}))) sq2 on sq1.foiministryrequestid = sq2.foiministryrequestid)
        left join

        (WITH ranked AS (
        SELECT ministryrequestid, get_json_object(feedata, '$.estimatedelectronicpages') AS estimatedelectronicpages,
        get_json_object(feedata, '$.estimatedhardcopypages') AS estimatedhardcopypages,
                ROW_NUMBER() OVER (PARTITION BY cfrfeeid ORDER BY version DESC) AS rn
        FROM foi_mod.foirequestcfrfees
        )
        SELECT *
        FROM ranked
        WHERE rn = 1) sq3 on sq3.ministryrequestid = sq1.foiministryrequestid

        join (

        select sq3.*, sq2.dedupepagecount from 
        (select foiministryrequestid, sum(pagecount) as pagecount from docreviewer.documents d
        group by foiministryrequestid) sq3
        join

        (select sum(pagecount) as dedupepagecount, foiministryrequestid from docreviewer.documents d1
        join 
        (select rank1hash, min(d.documentid) as docid  from docreviewer.documenthashcodes dhc
        join docreviewer.documents d on d.documentid = dhc.documentid
        group by rank1hash) sq on sq.docid = d1.documentid
        group by foiministryrequestid) sq2 on sq2.foiministryrequestid = sq3.foiministryrequestid) sq4 on sq4.foiministryrequestid = sq1.foiministryrequestid
        """

    print(query)

    df = spark.sql(query)
    df.show()


    # order of columns here is important!
    df_mapped = df.selectExpr(
        "foirequest_id AS foirequestid",
        f"{runcycleid} as runcycleid",
        "count AS noofpagesreviewed",
        "pagesreleased AS noofpagesreleased",
        "dedupepagecount AS noofpagesdeduplicated",
        "NULL AS noofpagesintherequest",
        "estimatedelectronicpages AS electronicpageestimate",
        "estimatedhardcopypages AS physicalpageestimate",
        "'' AS noofpagesinreviewlog",
        "'' AS noofpagesinredactionlayer",
        "'Y' AS activeflag",
    )
    df_mapped.show()
    df_mapped.write.format("delta").mode("append").option("mergeSchema", "false").insertInto("factrequestdocumentsdetails")  
    etl_helpers.end_run_cycle(runcycleid, 't', "factRequestDocumentsDetails")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestDocumentsDetails", "Credentials not available")
    raise Exception("notebook failed") from e
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', "factRequestDocumentsDetails")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestDocumentsDetails", f"An error occurred: {e}")
        raise Exception("notebook failed") from e

In [0]:
%sql
        select
          sq1.foiministryrequestid,
          count, 
          case when requeststatuslabel = 'closed' then count else 0 end as pagesreleased,
          sq2.created_at,
          sq2.updated_at,
          sq2.foirequest_id,
          estimatedelectronicpages,
          estimatedhardcopypages,
          pagecount,
          dedupepagecount
        from 
        (
          (
            select foiministryrequestid, count(*) as count
            from (
              select 
                EXPLODE(
                  FROM_JSON(
                      pageflag,
                      'ARRAY<STRUCT<flagid: INT, page: INT, programareaid: ARRAY<INT>, other: ARRAY<STRING>>>'
                  )
                ) as pageflags,
                -- explode(split(pageflag, ',')) as pageflags,
                -- pageflag, 
                dpf.foiministryrequestid
              from docreviewer.documentpageflags dpf
              join docreviewer.documents d on dpf.documentid = d.documentid
              join docreviewer.documentmaster dm on dm.documentmasterid = d.documentmasterid
              left join docreviewer.documentdeleted dd on dd.filepath || '%' ilike dm.filepath
              where dd.deleted is null or dd.deleted is false
            )
            group by foiministryrequestid
          ) sq1

          join (
            WITH ranked AS (
              SELECT
                *,
                ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version DESC) AS rn
              FROM foi_mod.FOIMinistryRequests fmr
            )
            SELECT ranked.foiministryrequestid, ranked.requeststatuslabel, ranked.created_at, ranked.updated_at, ranked.foirequest_id
            FROM ranked
            WHERE rn = 1 and (foiministryrequestid in ({comma_list}))
          ) sq2 on sq1.foiministryrequestid = sq2.foiministryrequestid
        )
        
        left join
        (
          WITH ranked AS (
            SELECT
              ministryrequestid,
              get_json_object(feedata, '$.estimatedelectronicpages') AS estimatedelectronicpages,
              get_json_object(feedata, '$.estimatedhardcopypages') AS estimatedhardcopypages,
              ROW_NUMBER() OVER (PARTITION BY cfrfeeid ORDER BY version DESC) AS rn
            FROM foi_mod.foirequestcfrfees
          )
          SELECT *
          FROM ranked
          WHERE rn = 1
        ) sq3 on sq3.ministryrequestid = sq1.foiministryrequestid

        join (
          select sq3.*, sq2.dedupepagecount
          from 
          (
            select foiministryrequestid, sum(pagecount) as pagecount from docreviewer.documents d
            group by foiministryrequestid
          ) sq3
          join
          (
            select sum(pagecount) as dedupepagecount, foiministryrequestid
            from docreviewer.documents d1
            join 
            (
              select rank1hash, min(d.documentid) as docid  from docreviewer.documenthashcodes dhc
              join docreviewer.documents d on d.documentid = dhc.documentid
              group by rank1hash
            ) sq on sq.docid = d1.documentid
            group by foiministryrequestid
          ) sq2 on sq2.foiministryrequestid = sq3.foiministryrequestid
        ) sq4 on sq4.foiministryrequestid = sq1.foiministryrequestid

In [0]:
%sql

WITH MinistryRequest AS (
    SELECT
        foiministryrequestid,
        requeststatuslabel,
        created_at,
        updated_at,
        foirequest_id
    FROM (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version DESC) AS rn
        FROM foi_mod.FOIMinistryRequests
        WHERE foiministryrequestid IN ({comma_list})
    )
    WHERE rn = 1
),

CfrFees AS (
    SELECT
        ministryrequestid,
        get_json_object(feedata, '$.estimatedelectronicpages') AS estimatedelectronicpages,
        get_json_object(feedata, '$.estimatedhardcopypages') AS estimatedhardcopypages
    FROM (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY cfrfeeid ORDER BY version DESC) AS rn
        FROM foi_mod.foirequestcfrfees
    )
    WHERE rn = 1
),

DocumentsData AS (
    SELECT
        d.foiministryrequestid,
        SUM(d.pagecount) AS total_pagecount,
        SUM(CASE WHEN dhc.rank1hash IS NOT NULL THEN d.pagecount ELSE 0 END) AS dedupepagecount
    FROM docreviewer.documents d
    LEFT JOIN (
        SELECT
            documentid,
            rank1hash
        FROM docreviewer.documenthashcodes
        GROUP BY
            rank1hash, documentid
    ) AS dhc ON d.documentid = dhc.documentid
    GROUP BY
        d.foiministryrequestid
),

PageFlags AS (
    SELECT
        foiministryrequestid,
        COUNT(*) AS flag_count
    FROM (
        SELECT
            dpf.foiministryrequestid,
            EXPLODE(
                FROM_JSON(
                    pageflag,
                    'ARRAY<STRUCT<flagid: INT, page: INT, programareaid: ARRAY<INT>, other: ARRAY<STRING>>>'
                )
            ) AS pageflags
        FROM docreviewer.documentpageflags dpf
        JOIN docreviewer.documents d ON dpf.documentid = d.documentid
        JOIN docreviewer.documentmaster dm ON dm.documentmasterid = d.documentmasterid
        LEFT JOIN docreviewer.documentdeleted dd ON dd.filepath LIKE CONCAT(dm.filepath, '%')
        WHERE dd.deleted IS NULL OR dd.deleted IS FALSE
    )
    GROUP BY
        foiministryrequestid
)

SELECT
    mr.foiministryrequestid,
    pf.flag_count,
    CASE
        WHEN mr.requeststatuslabel = 'closed' THEN pf.flag_count
        ELSE 0
    END AS pagesreleased,
    mr.created_at,
    mr.updated_at,
    mr.foirequest_id,
    cf.estimatedelectronicpages,
    cf.estimatedhardcopypages,
    dd.total_pagecount AS pagecount,
    dd.dedupepagecount
FROM MinistryRequest mr
LEFT JOIN PageFlags pf ON mr.foiministryrequestid = pf.foiministryrequestid
LEFT JOIN DocumentsData dd ON mr.foiministryrequestid = dd.foiministryrequestid
LEFT JOIN CfrFees cf ON mr.foiministryrequestid = cf.ministryrequestid;